In [21]:
import sys

import json
import pickle
import pandas
import torch
import asyncio
import numpy as np 
import pandas as pd
import seaborn as sns   
from matplotlib import pyplot as plt
sys.path.append("/home/v-oostapenko/dev/mttl")    
from scipy.stats import entropy as calc_entropy
from mttl.cluster_tuning.encodings import ClusterInfos
from mttl.datamodule.alpaca_data_module import AlpacaDataModule
from finetune_llama import Config
from mttl.cluster_tuning.cluster_reader import ClusterResult
from inst_follow.utils import load_model, TopicRouter,disable_torch_init
device = "cuda" if torch.cuda.is_available() else "cpu"
# import matplotlib.pyplot as plt
# import seaborn as sns

import json
import os 
import sys 
import numpy as np
import click
import json
import tqdm
import copy
import torch
import datasets
from types import SimpleNamespace
sys.path.append("/home/v-oostapenko/dev/mttl")
from inst_follow.models.clm import CLM  
from transformers import LlamaTokenizer  
from mttl.models.poly import get_selector     
from mttl.models.modify_model import modify_transformer  
from finetune_llama import parse_config, Config
from inst_follow.utils import load_model, TopicRouter,disable_torch_init
from mttl.cluster_tuning.cluster_reader import ClusterResult
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
import openai
openai.api_key = "f48b5a4f15dc4e58991738ab066ba465"

In [ ]:
# evaluate each 
df = AlpacaDataModule(Config())
dataset=df.get_dataset()
def get_samples(dataset, idxs):
    samples = []
    gt_responses = []
    for i in idxs:
        sample = dataset[i].input_text
        #discard everything after \n### Response:
        sample, response = sample.split("\n### Response:")
        sample+="\n### Response:"
        samples.append(sample)
        gt_responses.append(response)
    return samples, gt_responses

In [ ]:
skill_set = ["writing",
             "roleplay", 
             "comprehension", 
             "common-sense reasoning", 
             "coding", 
             "math", 
             "factual knowledge", 
             "conterfactual reasoning", 
             "navigation",
             "planning",
             "social skills",
             "communication",
             "coordination",
             "negotiation",
             "leadership",]

skill_set = (
    "invent",
    "solve",          
    "factual knowledge",
    "common-sense reasoning",
    "reflect",
    "communicate",
    "categorize",
    "plan",
    "program/code",
    "math", 
    "write",
    "reading comprehension",
    "summarize",
    "storytell",
    "explain",
)
skill_set=set(skill_set)
def construct_gpt_template(example:dict):  
    prompt = f"I am trying to cluster instructions/questions based on which skill is needed to answer them. \
                \n To do this, given an instruction, please provide a comprehensive set of skills needed to answer it. Here is an incomplete set of possible skills needed to answer this instruction: {[skill for skill in skill_set]}. Feel free to use skills not in the list.\
                For each skill you list, please provide a score from 0 to 1 indicating how important that skill is to answer the instruction, the scores of all skills together must sum to 1.\
                \n First, think step by step why would you need each of the skills. At the end of your output provide a comprehensive set of skills as a JSON-formatted dictionary, where the keys are the skills and values are the scores for the following instruction:\
                \n### Instruction: {example['instruction']}."
    if example['input']:
        prompt+=f" ### Input: {example['input']}."
    prompt+="\n### Skills:"
    return prompt

In [ ]:
openai.api_type = "azure"
openai.api_base = "https://gcrgpt4aoai7.openai.azure.com/"
openai.api_version = "2023-05-15"

In [ ]:
from inst_follow.operator import GPT

In [ ]:
operator = GPT(model_name="gpt-4")   
inputs = [construct_gpt_template(example) for example in dataset.dataset]

In [ ]:
task = asyncio.create_task(operator.gather_chat_response(inputs[:2]))

In [ ]:
output = await task

In [ ]:
for inp in tqdm.tqdm(range(0, len(inputs), 10)):
    print(inp)

In [ ]:

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned")

In [ ]:
dataset['train']

In [ ]:
new_dataset = {
    "instruction": [],
    "input": [],
    "output": [],
    "skill_set": [],
}
import time
for i,example in tqdm.tqdm(enumerate(dataset.dataset)):
    if i==10:
        break
    template_to_fill = construct_gpt_template(example)
    print(f"PROMPT:\n{template_to_fill}")
    failure = True
    num_retries = 0        
    while failure and num_retries < 3:
        cache_row = None
        try:
            start = time.perf_counter()  
            response = openai.ChatCompletion.create(
                # model="gpt-3.5-turbo",                
                deployment_id="gpt-35-turbo",
                # model_name="gpt-4",
                messages=[
                    {"role": "user", "content": template_to_fill},
                ],
            )
            message = json.loads(str(response.choices[0]))["message"]["content"]
            print(message)
            try:
                # extract JSON from message
                json_start = message.find("{")
                json_end = message.find("}")   
                json_string = message[json_start:json_end+1]
                #parse JSON
                skills_dict = json.loads(json_string)
                skills_used = [skill.strip() for skill in skills_dict.keys() if skills_dict[skill]>0]
                # append new skills to skill set
                skill_set = set(list(skill_set)+skills_used)
                
                num_retries += 1
                failure = False
                new_dataset["instruction"].append(example["instruction"])
                new_dataset["input"].append(example["input"])
                new_dataset["output"].append(example["output"])
                new_dataset["skill_set"].append(skills_dict)
            except:                
                num_retries += 1
                
            end = time.perf_counter()
            if end - start < 1:
                time.sleep(1 - (end - start))
        except:
            time.sleep(3)

In [ ]:
with open("new_dataset.json", "w") as f:
    json.dump(new_dataset, f)

In [ ]:
"factual_knowledge".strip().replace("_", " ")

### Create cluster files fore specialized topics created by GPT4

In [45]:
from nomic import atlas
import json
import sys
import tqdm
import numpy as np
sys.path.append("/home/v-oostapenko/dev/mttl")
from mttl.utils import hash_example
from mttl.dataloader.alpaca_dataset_readers import AlpacaTemplateForHash as AlpacaTemplate
from mttl.cluster_tuning.encodings import ClusterInfos

In [95]:
# load dataset    
                  
with open("data/SI_30k_GPT4_clust.jsonl", "r") as f:
    new_dataset = json.load(f)

In [96]:
new_dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'gpt_output': 'Instruction 0: Give three tips for staying healthy.\n\nFor this instruction, the main action required is to "list" three tips that are relate

In [97]:
from collections import defaultdict
topics = defaultdict(list)
dataset = []  
for i,d in enumerate(new_dataset):
    # sample = {}
    # sample["instruction"] = d["instruction"]
    # sample["input"] = d["input"]
    # sample["output"] = d["output"]
    # sample["topics"] = " ".join(list(d["topic_set"].keys()))
    tt = d["skill_set"]
    tt = d["topic_set"]
    for t,v in tt.items():       
        if v>0.5:
            topics[t].extend([i])
    # dataset.append(sample)

In [98]:
large_topics = []
small_topics = []
for t in topics:
    # print(t, len(topics[t]))
    if len(topics[t]) > 500:
        large_topics.append(t)
    else:
        small_topics.append(t)

In [99]:
for t in large_topics:
    print(t, len(topics[t]))

math 1835
language 1706
computer science 3161
grammar 990
animals 928
literature 552
food 1110


In [92]:
testing_idxs = []
training_idxs = []
for t in large_topics:
    #shuffle topics[t]
    print(t)
    np.random.shuffle(topics[t])   
    training_idxs.extend(topics[t][:700])
    training_idxs.extend(topics[t][840:1590])
    testing_idxs.extend(topics[t][700:840])



In [58]:
len(testing_idxs)+len(training_idxs)

5040

In [64]:
dataset_of_6_topics = []
for i in training_idxs:   
    dataset_of_6_topics.append(new_dataset[i])
for i in testing_idxs:
    dataset_of_6_topics.append(new_dataset[i])
with open("data/SI_6_skills_5k.jsonl", "w") as f:
    json.dump(dataset_of_6_topics, f, indent=4)

In [68]:

cluster_infos = ClusterInfos() 
idxs={}
for i in tqdm.tqdm(training_idxs):
    example = new_dataset[i]
    # topics_example = example["skill_set"]
    topics_example = example["topic_set"]
    enc_input = AlpacaTemplate.apply(example)    
    hash = hash_example(enc_input)  
        
    probs = np.zeros((len(topics.keys())))
    for ii, k in enumerate(topics_example): 
        if topics_example[k]==1.:  
            idx_of_k_in_topics = list(topics.keys()).index(k) 
            probs[idx_of_k_in_topics]=topics_example[k]
            idxs[k]=idx_of_k_in_topics
            # print(k, idx_of_k_in_topics, topics_example[k])
            break
    assert np.sum(probs)==1
    # print(np.sum(probs))    
    main_t = np.argmax(probs)
    cluster_infos.is_test.extend([0])   
    cluster_infos.task_names.extend(" ".join(list(topics_example.keys())[0]))
    cluster_infos.cluster_ids.extend([int(main_t)])    
    cluster_infos.hashes.extend([hash])
    cluster_infos.cluster_dists.extend([probs.tolist()])
for i in tqdm.tqdm(testing_idxs):
    example = new_dataset[i]
    # topics_example = example["skill_set"]    
    topics_example = example["topic_set"]
    enc_input = AlpacaTemplate.apply(example)    
    hash = hash_example(enc_input)  
        
    probs = np.zeros((len(topics.keys())))
    for i, k in enumerate(topics_example): 
        if topics_example[k]==1:
            idx_of_k_in_topics = list(topics.keys()).index(k) 
            probs[idx_of_k_in_topics]=topics_example[k]
            break
    assert np.sum(probs)==1
    # print(np.sum(probs))    
    main_t = np.argmax(probs)  
    cluster_infos.is_test.extend([1])   
    cluster_infos.task_names.extend(" ".join(list(topics_example.keys())))
    cluster_infos.cluster_ids.extend([int(main_t)])    
    cluster_infos.hashes.extend([hash])
    cluster_infos.cluster_dists.extend([probs.tolist()])

100%|██████████| 840/840 [00:00<00:00, 6280.83it/s]


In [69]:
idxs

{'math': 5,
 'language': 7,
 'computer science': 8,
 'grammar': 15,
 'animals': 18,
 'food': 31}

In [153]:
hash_to_idx_path="/home/v-oostapenko/dev/mttl/inst_follow/data/hash_to_idx_si.json"
with open(hash_to_idx_path, "r") as f:
                    hash_to_idx = json.load(f)

In [154]:
cluster_infos.hashes[0]

'afbd51db7f2e597e5fb551871ce002a4'

In [155]:
for i,hash in enumerate(cluster_infos.hashes):
    if hash not in hash_to_idx:
        print(f"hash {i}, {hash} not in hash_to_idx")

hash 1713, d7ed467861871816805aa6c3884c1c4f not in hash_to_idx
hash 1778, b299b97b87efdf0e552dc59a201fe3c0 not in hash_to_idx


In [156]:
hash_to_idx["45af3def42ed8270b2e16d6e42611be8"]

27527

In [130]:

idx = training_idxs[423]
example = new_dataset[idx]
topics_example = example["topic_set"]
enc_input = AlpacaTemplate.apply(example)    
hash = hash_example(enc_input)  

In [138]:
enc_input

'Below is an instruction that describes a task. Write a response that appropriately completes the request.            \n### Instruction: Rewrite the formula $y = \\frac{2x-1},{3}$ so that it can be solved for x.            \n### Response: The formula $y = \\frac{2x-1},{3}$ can be rewritten to solve for x as follows:\n\nFirst, we start by isolating the term involving x on one side of the equation. To do this, we multiply both sides by 3:\n\n$3y = \\frac{2x-1},{3}\\times 3$\n\n$3y = 2x-1$\n\nNext, we add 1 to both sides to isolate the 2x term:\n\n$3y + 1 = 2x -1 + 1$\n\n$3y + 1 = 2x$\n\nFinally, we divide both sides by 2 to solve for x:\n\n$\\frac{3y+1},{2} = \\frac{2x},{2}$\n\n$x =\\frac{3y+1},{2}$\n\nSo the formula $y = \\frac{2x-1},{3}$ can be rewritten to solve for x as $x = \\frac{3y+1},{2}$.'

In [60]:
cluster_probs=np.array(cluster_infos.cluster_dists)
cluster_probs=cluster_probs[:,cluster_probs.sum(0).nonzero()].squeeze()

In [62]:
cluster_probs.shape

(5040, 7)

In [63]:
skill_ids_to_keep = np.where((np.array(cluster_infos.cluster_dists).sum(0))> 0)[0]  
for i,d in enumerate(cluster_infos.cluster_dists):
    cluster_infos.cluster_dists[i] = np.array(d)[skill_ids_to_keep].tolist()
print(np.array(cluster_infos.cluster_dists).shape)

(5040, 7)


In [68]:
example_to_ids_path="/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/gpt_6_specialized_skills.pkl"
cluster_infos.save(example_to_ids_path)

In [ ]:
# save dataset of 6 topics

# Dolly

In [12]:
from mttl.dataloader.db_dolly_ds_reader import DB_DollyDataset
from datasets import load_dataset

In [13]:
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-7b-hf", add_eos_token=True)
tokenizer.pad_token_id = 0 
max_input_length=256   
max_output_length=128 
data_dir=None               

In [14]:
dolly_ds = DB_DollyDataset(tokenizer, max_input_length, max_output_length, data_dir, train_on_inputs=False)

100%|██████████| 1/1 [00:00<00:00, 832.20it/s]


In [15]:
dataset = load_dataset("databricks/databricks-dolly-15k")["train"] 

100%|██████████| 1/1 [00:00<00:00, 1028.52it/s]


In [16]:
dataset[0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

In [17]:
# topic idxs
from collections import defaultdict
topics = defaultdict(list)
for i,ex in enumerate(dataset):
    topics[ex["category"]].append(i)
rng = np.random.default_rng(1234)
test_idxs = []        
for t,v in topics.items():
    ti = rng.choice(topics[t], size=100, replace=False)
    test_idxs.extend(ti)
    print(t, len(ti), len(v))

closed_qa 100 1773
classification 100 2136
open_qa 100 3742
information_extraction 100 1506
brainstorming 100 1766
general_qa 100 2191
summarization 100 1188
creative_writing 100 709


In [18]:

cluster_infos = ClusterInfos() 
idxs={}
for i,example in tqdm.tqdm(enumerate(dolly_ds)):
    # topics_example = example["skill_set"]
    topics_example = example.category
    hash = example.hash        
    probs = np.zeros((len(topics.keys())))           
    topic_id = list(topics.keys()).index(topics_example)
    probs[topic_id]=1
    is_test = int(i in test_idxs)
    assert np.sum(probs)==1
    # print(np.sum(probs))    
    main_t = np.argmax(probs)
    cluster_infos.is_test.extend([is_test])     
    cluster_infos.task_names.extend([topics_example])
    cluster_infos.cluster_ids.extend([int(main_t)])    
    cluster_infos.hashes.extend([hash])
    cluster_infos.cluster_dists.extend([probs.tolist()])

15011it [00:19, 753.32it/s]


In [19]:
np.array(cluster_infos.cluster_dists).sum(0)

array([1773., 2136., 3742., 1506., 1766., 2191., 1188.,  709.])

In [20]:
example_to_ids_path="/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/db_dolly.pkl"
cluster_infos.save(example_to_ids_path)

# Filter SI dataset

In [ ]:
# Training set:
# - only low entropy
# - both, low and high
# for both we need test examples with low entropy, i.e. per cluster
# for both we need train examples with high entropy

In [23]:

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned")
# clsuter results
from mttl.cluster_tuning.cluster_reader import ClusterResult


100%|██████████| 1/1 [00:00<00:00, 753.02it/s]


Bad pipe message: %s [b'\xfe\xf4\xcdv\xf5Qq\x02\xde<\t\'\x97\r\x9e\x1d\xf5\x1a \x9f\x05\xd1\x04.\xf4\xf12\xdf"\xc5S\xa3T^\x83\xb6\x0e\xae\xb3to\xa5\x9a\xe9\xe8a=|\xb2\x8c\x9e\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01', b'']
Bad pipe message: %s [b'\x1b\xd3\xdb\xd0\xa9\xd0\xcaat\xf7\xe8R\xcd<\xcd\xf5\xf4t \xe2\xef\xa0\xc3\x0f<c\xcf\xe5\xccm\x11yF)\xd0\xb9\x13\xa2#\xcf\x89)T\x9ed\xd4\nG}f\xb8\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\